In [1]:
import numpy as np
import pandas as pd
import os
import collections
import random
import itertools
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [2]:
# Get the feature space dimension so we can initialize the data matrix
kmer_cnts = collections.defaultdict(lambda:0)
genome_files = [file_nm for file_nm in os.listdir('./pen_cnts') if file_nm !="c.py"]


In [ ]:
#optional to run, gets number of kmers but not necessary for this code
for file_nm in genome_files:
    with open('./pen_cnts/' + file_nm, 'r') as fd:
        kmer_ids = set(fd.readlines()[0].rstrip().split(' '))
        for k_id in kmer_ids:
            kmer_cnts[k_id] += 1
            
print('The number of kmers is {}'.format(len(kmer_cnts)))

Functions to create matrix from jellyfish results and desired labeling of genomes.

In [3]:
def createMatrix(mat, genome_files):
    for row_i, file_nm in enumerate(genome_files):
        with open('./pen_cnts/' + file_nm, 'r') as fd:
                 kmer_ids = set(fd.readlines()[0].rstrip().split(' '))
                 for k_id in kmer_ids:
                    mat[row_i, int(k_id)] = 1
    return mat

In [4]:
def createLabels(labels, genome_files):    
    for row_i, file_nm in enumerate(genome_files):
        genome_id = file_nm.partition('.PATRIC')[0]
        pheno = truth_dat[truth_dat['genome_id'] == genome_id].iloc[0]['resistant_phenotype']
        if pheno == 'Susceptible' or pheno == 'Intermediate':
             labels[row_i, 0] = 1
    return labels

Get data from original tsv for penicillin to use when creating labels.

In [5]:
truth_dat = pd.read_csv("PATRIC_genomes_AMR.tsv", sep='\t', dtype=str)
truth_dat = truth_dat[truth_dat['antibiotic'] == 'penicillin']

In [6]:
size =len(genome_files)

Create matrix of jellyfish results using createMatrix function.

Can skip and load compressed matrix later.

In [11]:
matrix = np.zeros([size, 30000000], dtype=np.int8)

In [12]:
matrix = createMatrix(matrix, genome_files)

In [8]:
print(matrix.shape)

(299, 30000000)


Delete columns for kmers that no genome has.

Can skip and load compressed matrix later.

In [4]:
sums = matrix.sum(axis=0)
lsums = sums.tolist()
indexes = list(range(len(lsums)))


In [8]:
cols =[ _ for _ in itertools.compress(indexes, map(lambda x: x>=1,lsums)) ]

In [13]:
_matrix = matrix[:,cols]
print(_matrix.shape)

(299, 2094629)


Get desired labels for all the genomes.

In [31]:
labels = np.zeros([size, 1], dtype=np.int8)

In [36]:
labels = createLabels(labels,genome_files)

Alternatively you can load in the labels from when this was run before to save time. 

In [9]:
labels = np.load('labelsss.npy')

Reshape matrix and get sums of every 193 columns to reduce feature space. (Used 193 and 10853 since they are factors of 2094629.

Can skip and load compressed matrix in next step.

In [15]:
mat = _matrix.reshape(-1, 10853, 193).sum(axis=2)

You can skip all the matrix related prior steps and load the reduced matrix here from past runs.

In [10]:
mat = np.load('mat.npy')

In [11]:
print(mat.shape)

(299, 10853)


Run cross validation with default AdaBoost Classifier and determine accuracy.

In [13]:
clf1 = AdaBoostClassifier()
scores1 = cross_val_score(clf1, mat, np.ravel(labels), cv=10)

In [17]:
print(scores1)
print(np.mean(scores1))

[ 0.83333333  0.93333333  0.8         0.83333333  0.8         0.8
  0.83333333  0.66666667  0.83333333  0.51724138]
0.785057471264


Run cross validation with Random Forest Classifier and find accuracy.

In [15]:
clf2 = RandomForestClassifier(n_jobs=16, max_features="sqrt", n_estimators=50)
scores2 = cross_val_score(clf2, mat, np.ravel(labels), cv=10)

In [18]:
print(scores2)
print(np.mean(scores2))

[ 0.86666667  0.8         0.86666667  0.9         0.76666667  0.83333333
  0.8         0.86666667  0.83333333  0.5862069 ]
0.811954022989


Run cross validation with Decision Tree Classifier and calculate accuracy.

In [20]:
clf3 = DecisionTreeClassifier()
scores3 = cross_val_score(clf3, mat, np.ravel(labels), cv=10)

In [21]:
print(scores3)
print(np.mean(scores3))

[ 0.76666667  0.76666667  0.76666667  0.83333333  0.66666667  0.83333333
  0.76666667  0.83333333  0.66666667  0.4137931 ]
0.731379310345


Below is an example of how to execute without cross validation but with random divisions with 1/3 set aside as the testing set. 
First split the data:

In [18]:
train_rows = random.sample(range(299), 200)

In [24]:
test_rows = set(range(299))-set(train_rows) 

In [38]:
test_mat = mat[list(test_rows),:]

In [39]:
test_labels = labels[list(test_rows),:]

In [40]:
train_mat = mat[train_rows,:]
train_labels = labels[train_rows,:]

Run classifier on split data and get accuracy.

In [47]:
clf = AdaBoostClassifier()
clf.fit(train_mat, np.ravel(train_labels))
adapreds = clf.predict(test_mat)

In [49]:
accuracy_score(adapreds, test_labels)

0.81818181818181823